In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Load environment variables in a file called .env

load_dotenv()
openai=OpenAI()

In [3]:
MODEL="gpt-4o-mini"

In [4]:
system_message="You are a helpful assistant"

Reminder of the structure of prompt messages to OpenAI: <br /> 
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]<br /> 
We will write a function chat(message, history) where: message is the prompt to use, history is a list of pairs of user message with assistant's reply. Gradio feed history in below format
<br /> 
[
    ["user said this", "assistant replied"],
    ["then user said this", "and assistant replied again],
    ...
]
<br /> 
We will convert this history into the prompt style for OpenAI, then call OpenAI.

In [16]:
def chat(message, history):
    # messages=[{"role":"system","content":system_message}]
    # for user_msg,ai_msg in history:
    #     messages.append({"role":"user","content":user_msg})
    #     messages.append({"role":"assistant","content":ai_msg})
    # messages.append({"role":"user","content":message})
    print(history)
    
    history.append({"role":"user","content":message})
    s=openai.chat.completions.create(
        messages=history,
        model=MODEL,
        stream=True
    )
    r=""
    for c in s:
        r+=c.choices[0].delta.content or ''
        yield r

In [17]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


[]
[{'role': 'user', 'metadata': {'title': None}, 'content': 'hi'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': 'Hello! How can I assist you today?'}]
[{'role': 'user', 'metadata': {'title': None}, 'content': 'hi'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'metadata': {'title': None}, 'content': 'Tell me a joke'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': 'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'}]
[{'role': 'user', 'metadata': {'title': None}, 'content': 'hi'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'metadata': {'title': None}, 'content': 'Tell me a joke'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': 'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'}, {'role': 'user', 'metadata': {'title': None}, 'content'

In [18]:
#Below is one shot prompting where we gave example to LLM
system_message="You are great sales executive at fashion store. All items except shoes are on 50 % sale and hats are on 60%\
discount. you should gently encourage customer to buy items on sale esp. hats. And if customer is not sure what to \
buy encourage them to buy hats. For example, customer asks 'What is best thing to buy', then you can say all things \
are wonderful if you waant to have best deal buy hats."

In [21]:
def chat(user_input,past):
    past.append({"role":"system","content":system_message})
    past.append({"role":"user","content":user_input})
    s=openai.chat.completions.create(
        messages=past,
        model=MODEL,
        stream=True
    )
    r=""
    for c in s:
        r+=c.choices[0].delta.content or ''
        yield r
    

In [22]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [25]:
#old style
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})

    if 'belt' in message:
        messages.append({"role": "system", "content": "For added context, the store does not sell belts, \
but be sure to point out other items on sale"})
    
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [26]:
gr.ChatInterface(fn=chat).launch()

/home/vboxuser/repos/llm/edDonner/venv/lib/python3.10/site-packages/gradio/components/chatbot.py:228: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
